## Expert python by Tech with Tim

  ##### Dunder Methods  or Data Model Methods 

In [ ]:
class Person:
    def __init__(self,person):
        self.person = person
    def __repr__(self):
        return "Person({})".format(self.person)
    def __call__(self,y):
        return self.person*y
    def __len__(self):
        return len(self.person)
        
p  = Person("Tim")
print(p)
print(p(4))
print(len(p))

In [ ]:
### Another example using Queue class since it doesnt have __repr__ methos
import inspect

from queue import Queue as q

class Queue(q):
    def __repr__(self):
        return "Queue({})".format(self._qsize())
    
    def __add__(self,args):
        if type(args)==list:
            for arg in args:
                self.put(arg)
        else:
            self.put(args)
    
    def __call__(self):
        return self.get()


        
qt = Queue()
qt+2
qt+4
qt+[3,4]
while qt.qsize() > 0 :
    print(qt.get())

### Metaclasses and how classes actually works ( They are complicated stuff tho)

In [ ]:
## metaclasses create rules for a new class
## Everything in python is an object 
"""Every new class in python is created using the type class
    We can literally work with type class to create new classes and
    add attributes and add base classes
"""



class Test:
    pass
print(type(Test()))

class Foo:
    def __init__(self,atr):
        self.atr = atr

        
        
def add_attr(self):
    self.z = 9
        
Test1 = type("Test",(Foo,),{"x":5, "add_attr":add_attr})
t = Test1("hello")
t.wy = "hi"
print(t.x)
print(t.wy)
print(t.atr)
t.add_attr()
print(t.z)


## I can argue that Test 1 and Test are exactly the same class and can do exactly the same things
## SO we can actually create a new class using type class by passin Name baseclass an

In [ ]:
########  Gettin into metaclasses
### Usually metaclassees inherit from type but necessatily
## __new__() dunder method is called before init methos and can be used to change how constructors are created
## We use the same args as the type class class_name, bases , attrs

## Metaclasses can be very powerful bcz we can hook onto the 
## construction of a class and change how a class is created

## Metaclasses can be magic . We just showed an example where we changes all the attributes to 
##  Upper Case 

## We use it while writing library code to enforece the inheritance from user class


class Meta(type):
    def __new__(self, class_name, bases, attrs): 
        print(attrs)
        
        a= {}
        for items in attrs:
            if items.startswith("__"):
                a[items] = attrs[items]
            else:
                a[items.upper()] = attrs[items]
        return type(class_name,bases,a)
    
    
        
class Dog(metaclass = Meta):
    
    x=5
    y=8
    
    def hello(self):
        print("Hi!!")
        
d  = Dog()
dir(Dog)

print(d.X)

### Decorators  are wrappers around the functions
##### You can modify the funciton without touching the funciton code

In [ ]:
##  We look into a case where we have two different functions func2 and func3
## func2 takes one argument and func3 takes 0 argument
## We need to apply the same decorator to both the function.  
## So we use *args and **kwargs in our decorator wrapper

def func(f):
    def wrapper(*args, **kwargs):
        print("Before func")
        rv = f(*args, **kwargs)
        print("After func")
        return rv
    return wrapper

@func
def func2(x,y):
    print("Func2 is {}".format(x))
    return y
    


@func
def func3():
    print("Heelo FUnc3")
    
func3()
print("="*30)
y = func2(34,23)
print(y)

In [ ]:
### lets make a time wrapper that times the run time of a funciton

import time 

def timer(func):
    def wrapper(*args,**kwargs):
        start = time.time()
        rv = func(*args,**kwargs)
        end = time.time()
        print("Timing function ======  {} ===== Time taken >>>>>> {}".format(func.__name__,end-start))
        return rv
    return wrapper

@timer
def test(num):
    for _ in range(num):
        continue

@timer
def test2():
    time.sleep(2)
    

test(100000)

test2()

 #  

#  

#### Generators in python

  ##### Problem that generators solve:
  ##### We have a finite memory(ram) in computer , we cant load everything in our program into memory
  ##### Generators are useful in this case

In [6]:
###Example  x = [i**2 for i in range(1000000000000)]  
### Dont run abovething bcz that will eat up the entire memory of your computer
### And you will have to wait for the entire program to complete
## We create our own generators

class Gen:
    def __init__(self,n):
        self.n = n
        self.last = 0
        
    def __next__(self):
        return self.next()
    
    def next(self):
        if self.last==self.n:
            raise StopIteration()
        rv = self.last**2
        self.last+=1
        return rv
    
g = Gen(10)
        
while True:
    try:
        next(g)
    except StopIteration:
        break
    

import sys

    
"""PYTHON HAS SIMPLIFIED THE GENERATOR SYNTAX FOR US
    We can use yield for it
"""
def gen(n):
    """When This function is called 
       The for loop runs and at the yield command
       it pauses the execution
    """
    for i in range(n):
        yield i**2
        
g1 = gen(1000000)

g2 = (i**2 for i in range(1000000)) 

list1 = [i**2 for i in range(1000000)]
"""Now when we run this wes will never run into a memory 
   problem as the entire data is never loaded into memory.
"""
print(sys.getsizeof(list1))

print(sys.getsizeof(g1))





8697464
120


#  

### Context Managers

  ##### To create a context manager you need to have two special dunder methods \_\_enter\_\_ and \_\_exit\_\_

In [4]:
### We use it all the time but dont notice it
## Example

""" So we open a file.txt file and do some stuff and close it
    But if we encounter a problem during the writing then
    we will not reach the closing statement . And the file will 
    still contibue to reside in memory
"""
file  = open("file.txt","w")
file.write("hello")
file.close()


"""So context managers help to solve this problem of opening one thing and closing it
   no matter what happens in between this two step
"""

### One possible solution is to use try and finally
file  = open("file.txt","w")
try:
    file.write("hello")
finally:
    file.close()
    

""" Using with we create a context manager . This is the most usual way.
    open('file.txt','w') this expressions tell the context manager what
    to do 
    
"""
with open('file.txt','w') as f:
    f.write("hello word")


""" Lets write a context manager as a class
"""

class File:
    
    def __init__(self,file,method):
        self.file = open(file,method)
        
    def __enter__(self):
        """ This enter method is the first thibg that happens
            Even before the init
        """
        print("Enter")
        return self.file
    
    def __exit__(self, type, value, traceback):
        """ If something happens in between then 
            this exception is handled here
        """
        print("Exit")
        self.file.close()
        

with File("file.txt","w") as f:
    print("Middle")
    f.write("Hello there somebody there")
    raise Exception()
        

Enter
Middle
Exit


Exception: 